<a href="https://colab.research.google.com/github/YasminHeimann/Graph_Anomaly/blob/master/DGL_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Basic DGL tutorial, training a 2-layer GCN**



In [3]:
# pip install dgl


In [5]:
print("finished dgl finished dgl installation")

finished dgl finished dgl installation


In [3]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F


Using backend: pytorch


Get dataset from DGL for Node Classification

In [4]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('\nNumber of categories:', dataset.num_classes)
# cora db has one graph
g = dataset[0]
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Extracting file to C:\Users\avdar\.dgl\cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.

Number of categories: 7
Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


**Define GCN model**

In [5]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = dgl.nn.GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    
# Create the model with given dimensions
print("features dim: ", g.ndata['feat'].shape[1])
# model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)


features dim:  1433


In [6]:

def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))


In [7]:
#g = g.to('cuda')
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes) #.to('cuda')
train(g, model)
print("\nfinished training GCN")

In epoch 0, loss: 1.946, val acc: 0.168 (best 0.168), test acc: 0.176 (best 0.176)
In epoch 5, loss: 1.885, val acc: 0.610 (best 0.610), test acc: 0.630 (best 0.630)
In epoch 10, loss: 1.797, val acc: 0.676 (best 0.694), test acc: 0.702 (best 0.719)
In epoch 15, loss: 1.685, val acc: 0.690 (best 0.694), test acc: 0.715 (best 0.719)
In epoch 20, loss: 1.548, val acc: 0.700 (best 0.702), test acc: 0.715 (best 0.716)
In epoch 25, loss: 1.389, val acc: 0.716 (best 0.720), test acc: 0.736 (best 0.726)
In epoch 30, loss: 1.216, val acc: 0.726 (best 0.726), test acc: 0.752 (best 0.752)
In epoch 35, loss: 1.037, val acc: 0.736 (best 0.736), test acc: 0.750 (best 0.748)
In epoch 40, loss: 0.863, val acc: 0.746 (best 0.746), test acc: 0.756 (best 0.756)
In epoch 45, loss: 0.704, val acc: 0.744 (best 0.748), test acc: 0.764 (best 0.760)
In epoch 50, loss: 0.566, val acc: 0.756 (best 0.756), test acc: 0.774 (best 0.774)
In epoch 55, loss: 0.451, val acc: 0.766 (best 0.766), test acc: 0.782 (best 0

In [8]:
print("hello gur")

hello gur
